# Load all survey data and merge into a single dataset

We'll load all the data downloaded in [download.ipynb](download.ipynb) and merge them into an `xarray.Dataset`, including relevant metadata. This can make the data more easily accessible if we want to use the entire thing. 

Not sure this will be a good idea since surveys are spread out in the globe. Let's see.

In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import pygmt as pg
import matplotlib.pyplot as plt
import verde as vd
import zipfile
from pathlib import Path
from tqdm import tqdm
from io import StringIO

In [2]:
output = Path("..") / "data"
output_raw = output / "raw"
output_processed = output / "intermediate"
if not output_processed.exists():
    output_processed.mkdir()

/home/leo/miniconda3/envs/noaa-aeromagnetic/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Convert the MAG88T files to CSV.

In [3]:
archive_name = output_raw / "admap_bas_post1990.zip"
for archive_name in tqdm(list(output_raw.glob("*.zip")), ncols=100):
    with zipfile.ZipFile(archive_name) as archive:
        data_files = [
            fname for fname in archive.namelist() 
            if "mag88t" in fname and fname.endswith(".m88t")
        ]
        assert len(data_files) == 1
        data_file = data_files[0]
        with archive.open(data_file) as survey_file:
            csv_fname = Path(data_file).parts[-1].replace(".m88t", ".csv")
            with open(output_processed / csv_fname, mode="w") as csv_file:
                for line in survey_file:
                    csv_file.write(line.decode().replace("\t", ","))

100%|█████████████████████████████████████████████████████████████| 131/131 [00:37<00:00,  3.49it/s]
